In [ ]:
# Copyright (c) 2022 - for information on the respective copyright owner see the NOTICE file or the repository https://github.com/boschresearch/the-atlas-benchmark
#
# SPDX-License-Identifier: Apache-2.0

# System imports

import yaml
from context import Benchmark, Evaluator
from context import Dataset
from context import Predictor_CVM
from context import Predictor_sof, Predictor_zan, Predictor_kara
import random
import numpy as np
from smac_optimizer import SmacOptimizer

Create the dataset object based on the yaml file configuration

In [ ]:
# Tested this instruction for the following datasets
# input_benchmark_cfg_path = '../cfg/dataset_config_hotel.yaml'
input_benchmark_cfg_path = '../cfg/dataset_config_eth.yaml'
# input_benchmark_cfg_path = '../cfg/dataset_config_thor1.yaml'
# input_benchmark_cfg_path = '../cfg/dataset_config_thor3.yaml'
# input_benchmark_cfg_path = '../cfg/dataset_config_atc.yaml'
# input_benchmark_cfg_path = '../cfg/dataset_config_test_traj.yaml'
# input_benchmark_cfg_path = '../cfg/dataset_config_test_point_obstacles.yaml'

with open(input_benchmark_cfg_path, 'r') as file:
    benchmark_cfg = yaml.load(file, Loader=yaml.FullLoader)

observation_len = benchmark_cfg['benchmark']['setup']['observation period']
prediction_horizon = benchmark_cfg['benchmark']['setup']['prediction horizon']

# Optional split parameter defines the portion of the dataset that is being extracted,
# frames between 0 * length(dataset) and 1 * length(dataset)
dataset_train = Dataset(benchmark_cfg, split=[0,0.2])
valid_scenes_train = dataset_train.extract_scenarios(prediction_horizon,observation_len,min_num_prople=2)
dataset_test = Dataset(benchmark_cfg, split=[0.2,1])
valid_scenes_test = dataset_test.extract_scenarios(prediction_horizon,observation_len,min_num_prople=2)

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

evaluation = Evaluator(set_GT_length=prediction_horizon)

#evaluation.evaluate_scenario_ade_fde(valid_scenes[19], predictions)
evaluation.plot_scenario(dataset_test, valid_scenes_train[11])

Preparing some predictors to be evaluated

In [ ]:
with open('../cfg/method_config_cvm.yaml', 'r') as file:
    method_cfg = yaml.load(file, Loader=yaml.FullLoader)
predictor_cvm = Predictor_CVM(dataset_train, method_cfg)

with open('../cfg/method_config_sof.yaml', 'r') as file:
    method_cfg = yaml.load(file, Loader=yaml.FullLoader)
predictor_sof = Predictor_sof(dataset_train, method_cfg)

Setting up the benchmark

In [ ]:
bench = Benchmark(verbose = False)
fde_value = bench.accuracy_experiment(valid_scenes_test, predictor_cvm, metric='fde')
print("Constant velocity model reaches FDE =", fde_value)
#bench.accuracy_experiment(valid_scenes_test, predictor_sof)

In [ ]:
input_method_cfg_path = '../cfg/method_config_kara.yaml'
e = SmacOptimizer(input_method_cfg_path, input_benchmark_cfg_path, dataset_train_split=[0,0.2], min_num_prople=2, iterations=300, metric='fde')
# !!! Be careful with this one, becaise it will overwrite the optimal values in the method_cfg file
# e.optimize()

In [ ]:
bench.verbose = True
with open(input_method_cfg_path, 'r') as file:
    method_cfg = yaml.load(file, Loader=yaml.FullLoader)

print("With default parameters the model reaches:")
predictor_default = Predictor_kara(dataset_train, method_cfg)
fde_value = bench.accuracy_experiment(valid_scenes_train, predictor_default, metric='fde')

print("With optimal parameters the model reaches:")
predictor_optimal = Predictor_kara(dataset_train, method_cfg, parameters=['optimal','eth'])
fde_value = bench.accuracy_experiment(valid_scenes_train, predictor_optimal, metric='fde')